In [1]:
from pyspark.sql import SparkSession
from delta import *

import subprocess
import json
import os

In [2]:
from uuid import uuid4

In [3]:
session_id = str(uuid4())

In [4]:
spark = SparkSession.builder.appName(session_id) \
        .master("spark://spark-master:7077")\
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.0")\
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
        .getOrCreate()
sqlContext = SparkSession(spark)
#Dont Show warning only error
spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/usr/local/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b075405b-849e-4da5-bcba-e9ef57a213a7;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.0 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.9.1 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0

In [ ]:
# builder = SparkSession.builder.appName(session_id) \
#     .master("spark://spark-master:7077")\
#     .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.0")\
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

# spark = configure_spark_with_delta_pip(builder).getOrCreate()    
# # spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/usr/local/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-11f5c8a5-f198-44c2-8fd5-9fa3707d0c92;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 555ms :: artifacts dl 21ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |  

In [5]:
from confluent_kafka.admin import AdminClient, NewTopic
from confluent_kafka import Producer, Consumer, KafkaError
from confluent_kafka.schema_registry import SchemaRegistryClient, Schema
from confluent_kafka import avro
import pyspark.sql.functions as pyspark_f
import pyspark.sql.types as pyspark_dt

# Create Topic & Schema Registry

In [7]:
topic_name = 'sample_topic'
kafka_config = {
    'bootstrap.servers': 'kafka-1:9092,kafka-2:9092,kafka-3:9092',
}

schema_registry_cofig =  {'url' : 'http://schema-registry:8085'} # URL for Schema Registry
schema_registry_subject = 'schema_registry_subject_example' 

In [55]:
# Create Kafka topic
admin_client = AdminClient(kafka_config)
new_topic = NewTopic(topic_name, num_partitions=2, replication_factor=2)

# Create the topic
admin_client.create_topics([new_topic])
print(f"Kafka topic {topic_name} created.")

Kafka topic sample_topic created.


In [62]:
producer = Producer(kafka_config)

In [64]:
# Produce messages
for i in range(10):
    key = str(i)
    value = {"name": f"User_{i}", "age": 20 + i}
    value_json = json.dumps(value)  # Convert dictionary to JSON string
    producer.produce(topic=topic_name, key=key, value=value_json)
    print(f"Produced Avro message: {value}")
producer.flush()

Produced Avro message: {'name': 'User_0', 'age': 20}
Produced Avro message: {'name': 'User_1', 'age': 21}
Produced Avro message: {'name': 'User_2', 'age': 22}
Produced Avro message: {'name': 'User_3', 'age': 23}
Produced Avro message: {'name': 'User_4', 'age': 24}
Produced Avro message: {'name': 'User_5', 'age': 25}
Produced Avro message: {'name': 'User_6', 'age': 26}
Produced Avro message: {'name': 'User_7', 'age': 27}
Produced Avro message: {'name': 'User_8', 'age': 28}
Produced Avro message: {'name': 'User_9', 'age': 29}


0

In [8]:
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_config['bootstrap.servers']) \
    .option("subscribe", topic_name) \
    .option("startingOffsets", "earliest") \
    .load()
